### library

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.dates as md
from matplotlib import pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import random

### Set_seed

In [6]:
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)
random.seed(random_seed)

torch.cuda.manual_seed(random_seed)

### get_train_array

In [10]:
train_array = np.load('./all_array_.npy')
print(train_array.shape)

inf_array = np.load('./inf_array_.npy')
print(inf_array.shape)

(58631, 2000, 10)
(8744, 2000, 10)


### loader_dataset
* first 3000 rows _train (this is for example)

In [26]:
BATCH_SIZE =  128
train_loader = torch.utils.data.DataLoader(train_array[:300,:,1:], batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
inf_loader = torch.utils.data.DataLoader(inf_array[:,:,1:], batch_size = BATCH_SIZE, shuffle=False, num_workers=0)

### model

**Encoder**

In [27]:
class Encoder(nn.Module):
    def __init__(self,  input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층


        self.lstm_1 = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size_1,
                      num_layers=self.num_layers, batch_first=True)

        self.lstm_2 = nn.LSTM(input_size=self.hidden_size_1, hidden_size=self.hidden_size_2,
                      num_layers=self.num_layers, batch_first=True)


    def forward(self, w):
        out_1, _ = self.lstm_1(w)

        return self.lstm_2(out_1)


**Decoder**

In [28]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층

        self.lstm_1 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=self.hidden_size_1,
                      num_layers=self.num_layers, batch_first=True)
        
        self.lstm_2 = nn.LSTM(input_size=self.hidden_size_1, hidden_size=self.input_size,
                      num_layers=self.num_layers, batch_first=True)


    def forward(self, z):
        out_1, _ = self.lstm_1(z)

        return self.lstm_2(out_1)

**Model_**

In [29]:
class UsadModel(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층
        
        self.encoder = Encoder(input_size, hidden_size_1, hidden_size_2, num_layers = 1)
        self.decoder1 = Decoder(input_size, hidden_size_1, hidden_size_2, num_layers = 1)
        self.decoder2 = Decoder(input_size, hidden_size_1, hidden_size_2, num_layers = 1)
        
    def forward(self, x, n):
        out_, _ = self.encoder(x)        
        out_.to(device)
        w1, _= self.decoder1(out_)
        w1.to(device)
        w2, _ = self.decoder2(out_)
        w2.to(device)
        self.encoder(w1)
        w3, _ = self.decoder2(self.encoder(w1)[0])
        w3.to(device)

        loss1 = 1/n*torch.mean((x-w1)**2)+(1-1/n)*torch.mean((x-w3)**2)
        loss2 = 1/n*torch.mean((x-w2)**2)-(1-1/n)*torch.mean((x-w3)**2)
        
        return loss1, loss2, out_, w2, w3

### get_model

In [30]:
device = 'cuda'
model = UsadModel(9,6,3)
optimizer1 = torch.optim.Adam(list(model.encoder.parameters())+list(model.decoder1.parameters()))
optimizer2 = torch.optim.Adam(list(model.encoder.parameters())+list(model.decoder2.parameters()))
model.to(device)

UsadModel(
  (encoder): Encoder(
    (lstm_1): LSTM(9, 6, batch_first=True)
    (lstm_2): LSTM(6, 3, batch_first=True)
  )
  (decoder1): Decoder(
    (lstm_1): LSTM(3, 6, batch_first=True)
    (lstm_2): LSTM(6, 9, batch_first=True)
  )
  (decoder2): Decoder(
    (lstm_1): LSTM(3, 6, batch_first=True)
    (lstm_2): LSTM(6, 9, batch_first=True)
  )
)

### train_function

In [31]:
# def train(model, train_loader, val_loader, epoch, optimizer1, optimizer2, device='cuda'):

def train(model, train_loader, epoch, optimizer1, optimizer2, device='cuda'):

    model.train()

    losses_train = []
    for batch in train_loader:
        loss1, loss2, out_,w2,w3 = model(batch.type(torch.FloatTensor).to(device),epoch+1)
        loss1.backward(retain_graph=True)
        loss2.backward()
        optimizer1.step()
        optimizer2.step()
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        losses_train.append([loss1.item(),loss2.item()])

    losses_train = np.array(losses_train)
    train_loss_1 = np.mean(losses_train[:,0])
    train_loss_2 = np.mean(losses_train[:,1])


    return train_loss_1, train_loss_2

### train

In [32]:
epochs=5
for epoch in range(epochs):
    train_loss_1, train_loss_2 = train(model, train_loader, epochs, optimizer1, optimizer2)
    print(f'epoch - {epoch}  train_loss_1 - {train_loss_1}  train_loss_2 - {train_loss_2}')

epoch - 0  train_loss_1 - 38.925638834635414  train_loss_2 - -25.948601722717285
epoch - 1  train_loss_1 - 38.922767639160156  train_loss_2 - -25.94760290781657
epoch - 2  train_loss_1 - 38.920056660970054  train_loss_2 - -25.946805318196613
epoch - 3  train_loss_1 - 38.91743405659994  train_loss_2 - -25.94622802734375
epoch - 4  train_loss_1 - 38.914856592814125  train_loss_2 - -25.945921262105305


### Inference

In [33]:
def testing(model, test_loader, alpha=.5, beta=.5, device = 'cuda'):
    results = np.empty([0, 9])
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.type(torch.FloatTensor).to(device)
            out_, _ = model.encoder(batch)
            w1, _ = model.decoder1(out_)
            w2, _ = model.decoder2(model.encoder(w1)[0])
            
            batch = batch.cpu()
            w1 = w1.cpu()
            w2 = w2.cpu()

            re_loss = alpha*torch.mean((batch-w1)**2, axis=1) + beta*torch.mean((batch-w2)**2, axis=1)
            re_loss = np.array(re_loss)
            
            results = np.concatenate([results,re_loss])

        return results

In [36]:
results = testing(model,inf_loader)
print(results)

array([[5.22976303e+00, 5.91227913e+00, 1.52015753e+01, ...,
        5.62714624e+00, 5.13415003e+00, 1.89419061e-01],
       [1.08160477e+01, 9.20279980e+00, 3.22795753e+01, ...,
        1.04262867e+01, 1.02415524e+01, 2.14587778e-01],
       [5.30385208e+00, 6.02832317e+00, 1.53928280e+01, ...,
        5.71127129e+00, 5.20741749e+00, 1.88655704e-01],
       ...,
       [7.62569666e+00, 7.11843538e+00, 2.21021156e+01, ...,
        6.62880373e+00, 7.83982038e+00, 1.95870474e-02],
       [7.29513550e+00, 6.93330956e+00, 2.13109665e+01, ...,
        6.49567556e+00, 7.50043011e+00, 1.82368327e-02],
       [1.81279297e+01, 2.12006378e+01, 4.14322205e+01, ...,
        1.99543285e+01, 9.87639427e+00, 1.54627132e+00]])